In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
data=pd.read_csv("Churn_modelling.csv")

In [ ]:
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [ ]:
data.head()

In [ ]:
Le1=LabelEncoder()
data.Gender=Le1.fit_transform(data.Gender)

In [ ]:
data.sample(5)

In [ ]:
# OHE
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse_output=False)
geo=ohe.fit_transform(data[['Geography']])

In [ ]:
ohe.get_feature_names_out(['Geography'])

In [ ]:
geo_df=pd.DataFrame(geo,columns=ohe.get_feature_names_out(['Geography']))

In [ ]:
geo_df

In [ ]:
data=pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)

In [ ]:
data.head()

In [ ]:
##saving encoders and scalers
with open('Label_Encoder_Gender.pkl','wb') as f:
    pickle.dump(Le1,f)

with open('Geo_OHE.pkl','wb') as f:
    pickle.dump(ohe,f)


In [ ]:
x=data.drop('Exited', axis=1)
y=data.Exited


In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.fit_transform(xtest)

In [ ]:
with open('sclaer.pkl','wb') as f:
    pickle.dump(scaler,f)


## ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(xtrain.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
log_dir='log/fit/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [ ]:
tcb=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
## setup early stopping
early_stopping= EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [ ]:
model.fit(xtrain,ytrain, 
        epochs=100,
        validation_data=(xtest,ytest),
        callbacks=[tcb,early_stopping])

In [ ]:
model.save('model.h5')

In [ ]:
## load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir log/fit
